In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

class YellowPagesScraper:
    def __init__(self, keyword, start_page=0, end_page=5, max_business=10, delay=1):
        """
        กำหนดค่าเริ่มต้นสำหรับการค้นหา
        keyword:       คำค้นหา (เช่น 'cafe')
        start_page:    หน้าแรกที่จะเริ่มค้นหา (เลขหน้า เริ่มที่ 0)
        end_page:      หน้าสุดท้ายที่จะค้นหา (ถ้ากำหนด จะไม่ใช้ max_business)
        max_business:  จำนวนธุรกิจสูงสุดที่ต้องการดึง (ถ้ากำหนด จะไม่ใช้ end_page)
        delay:         เวลาหน่วงระหว่าง request แต่ละรอบ (หน่วยวินาที)
        """
        self.keyword = keyword
        self.start_page = start_page
        self.end_page = end_page
        self.max_business = max_business
        self.delay = delay
        self.base_url = 'https://www.yellowpages.co.th'
        # รูปแบบ URL สำหรับค้นหา
        self.search_url_template = self.base_url + '/ypsearch?q={keyword}&page={page}'
        self.results = []  # เก็บผลลัพธ์ทั้งหมด

    def scrape(self):
        scraped = 0
        page = self.start_page
        while True:
            # หยุด loop ถ้าเลย end_page (ถ้ากำหนด)
            if self.end_page is not None and page > self.end_page:
                break
            # หยุด loop ถ้าครบ max_business (ถ้ากำหนด)
            if self.max_business is not None and scraped >= self.max_business:
                break

            # สร้าง url สำหรับค้นหาธุรกิจ
            search_url = self.search_url_template.format(keyword=self.keyword, page=page)
            print(f"Fetching page {page}: {search_url}")
            resp = requests.get(search_url)
            soup = BeautifulSoup(resp.content, 'html.parser')

            # หา div แต่ละอันที่เป็นรายการธุรกิจในหน้านี้
            listings = soup.find_all('div', class_='col-md-12 col-sm-12 col-xs-12 yp-search-listing no-gutter default-border margin-bottom-fifteen')
            if not listings:
                print("No more results.")
                break

            for listing in listings:
                # ถ้าครบจำนวนที่ต้องการแล้ว ให้ข้าม
                if self.max_business is not None and scraped >= self.max_business:
                    break
                # ดึง tag <a> ที่มี data-businessid (ชื่อธุรกิจและลิงก์)
                a_tag = listing.find('a', attrs={'data-businessid': True})
                name = a_tag.text.strip() if a_tag else ''
                link = a_tag['href'] if a_tag and a_tag.has_attr('href') else ''
                # เติม base_url ถ้า link ยังไม่สมบูรณ์
                if link and not link.startswith('http'):
                    link = self.base_url + link

                # เข้าไป scrape ข้อมูลรายละเอียดของแต่ละธุรกิจ
                address = ''
                phone = ''
                category = ''
                map_url = ''
                if link:
                    try:
                        profile_resp = requests.get(link)
                        profile_soup = BeautifulSoup(profile_resp.content, 'html.parser')
                        # Address: ดึงที่อยู่
                        addr_tag = profile_soup.find('div', class_='col-md-10 col-sm-10 col-xs-12 no-gutter')
                        address = addr_tag.text.strip() if addr_tag else ''
                        # Phone: ดึงเบอร์โทร
                        phone_tag = profile_soup.find('div', class_='col-md-10 col-sm-10 col-xs-12 no-gutter contact-details')
                        phone = phone_tag.text.strip() if phone_tag else ''
                        # Category: ดึงประเภทธุรกิจ (จาก <a> ภายใน div category)
                        cat_block = profile_soup.find('div', class_='col-md-12 col-sm-12 col-xs-12 no-gutter profile-content profile-category')
                        if cat_block:
                            cat_a = cat_block.find('a')
                            category = cat_a.text.strip() if cat_a else ''
                        # Map: ดึงลิงก์แผนที่ (ถ้ามี)
                        map_btn = profile_soup.find('a', class_='btn yp-white-btn-link')
                        map_url = map_btn['href'] if map_btn and map_btn.has_attr('href') else ''
                    except Exception as e:
                        print(f"Error scraping {link}: {e}")
                # เก็บข้อมูลธุรกิจในรูปแบบ dict
                self.results.append({
                    'Name': name,
                    # 'ProfileLink': link,
                    'Address': address,
                    'Phone': phone,
                    'Category': category,
                    'Map': map_url
                })
                scraped += 1
                print(f"Scraped {scraped}: {name}")
                time.sleep(self.delay)  # เวลาหน่วงระหว่าง request
            page += 1
        print(f"Scraping completed. {len(self.results)} profiles extracted.")

    def to_dataframe(self):
        # คืนค่าเป็น pandas DataFrame
        return pd.DataFrame(self.results)

    def to_csv(self, filename=None):
        # เซฟข้อมูลเป็นไฟล์ CSV
        if filename is None:
            filename = f"yellowpages_{self.keyword}.csv"
        df = self.to_dataframe()
        df.to_csv(filename, index=False)
        print(f"Saved to {filename}")

# Example usage:
# สร้าง object และสั่ง scrape ได้เลย (เลือกใช้งาน max_business หรือ end_page อย่างใดอย่างหนึ่ง)
# scraper = YellowPagesScraper(keyword='cafe', start_page=0, max_business=10)
# scraper = YellowPagesScraper(keyword='cafe', start_page=0, end_page=2)

# scraper.scrape()                 # ดึงข้อมูล
# df = scraper.to_dataframe()      # แปลงเป็น DataFrame
# print(df.head())                 # แสดงตัวอย่างข้อมูล

# scraper.to_csv()                 # เซฟเป็นไฟล์ CSV



In [4]:
# Define your list of keywords
keywords = ['cafe', 'bakery', 'car repair']  # Add as many as you want

for kw in keywords:
    print(f"\n=== Scraping for keyword: {kw} ===")
    # Choose either max_business or end_page (not both)
    scraper = YellowPagesScraper(keyword=kw, start_page=0, max_business=15)
    # If you want to use end_page instead, use: 
    # scraper = YellowPagesScraper(keyword=kw, start_page=0, end_page=2)

    scraper.scrape()
    # Save to CSV file, automatically named by keyword
    scraper.to_csv()
    print(f"Finished keyword: {kw}\n")


=== Scraping for keyword: cafe ===
Fetching page 0: https://www.yellowpages.co.th/ypsearch?q=cafe&page=0
Scraped 1: กัปตัน คาเฟ่ แอนด์ พีที คาร์วอช
Scraped 2: ร้านกาแฟ คาเฟ่ปิ่นเกล้า - Maka Cafe
Scraped 3: Tree Yoga & Cafe'
Scraped 4: SAYA cafe' & restaurant
Scraped 5: Part Cafe Chaiyaphum
Scraped 6: Phuket Moto Cafe
Scraped 7: Naka Loft Cafe
Scraped 8: Dog Cafe Bkk - CORGI IN THE GARDEN
Scraped 9: Howl 1.1.11 cafe
Scraped 10: white house cafe & Studio
Fetching page 1: https://www.yellowpages.co.th/ypsearch?q=cafe&page=1
Scraped 11: เดอะ เดวิส แบงคอก
Scraped 12: เครป คาเฟ่
Scraped 13: นานา กาแฟ
Scraped 14: เครป คาเฟ่
Scraped 15: ยูนีค มัลติ แบรนด์ คาเฟ่
Scraping completed. 15 profiles extracted.
Saved to yellowpages_cafe.csv
Finished keyword: cafe


=== Scraping for keyword: bakery ===
Fetching page 0: https://www.yellowpages.co.th/ypsearch?q=bakery&page=0
Scraped 1: Kidtueng Bakery Surin
Scraped 2: November Homemade Bakery
Scraped 3: ละมุนเค้ก Homemade Bakery
Scraped 4: KOLé Bakery B

,Name,ProfileLink,Address,Phone,Category,Map
0,กัปตัน คาเฟ่ แอนด์ พีที คาร์วอช,https://www.yellowpages.co.th/profile/%E0%B8%8...,116 ถนนโพนพิสัย ตำบลหมากแข้ง อำเภอเมืองอุดรธาน...,083-667-2728,ล้าง อัดฉีดและขัดมันรถยนต์,https://maps.google.com/maps/place?q=17.416769...
1,ร้านกาแฟ คาเฟ่ปิ่นเกล้า - Maka Cafe,https://makacafe.yellowpages.co.th,,,,
2,Tree Yoga & Cafe',https://www.yellowpages.co.th/profile/Tree-Yog...,61/2 ซอย ศุภราช 1 ถนน พหลโยธิน แขวงสามเสนใน เข...,093-636-1652,สอนโยคะ,https://maps.google.com/maps/place?q=13.784064...
3,SAYA cafe' & restaurant,https://www.yellowpages.co.th/profile/SAYA-caf...,54/5 หมู่ที่ 8 ตำบลในเมือง อำเภอเมืองชัยภูมิ ...,081-918-4640,ภัตตาคาร ร้านอาหารและสวนอาหาร,https://maps.google.com/maps/place?q=15.820506...
4,Part Cafe Chaiyaphum,https://www.yellowpages.co.th/profile/Part-Caf...,8 ตำบลในเมือง อำเภอเมืองชัยภูมิ จังหวัดชัยภูมิ...,083-378-2300,ผู้จำหน่ายและขายส่งกาแฟ,https://maps.google.com/maps/place?q=15.799229...
5,Phuket Moto Cafe,https://www.yellowpages.co.th/profile/Phuket-M...,75/23 ถนนบางกอก ตำบลตลาดเหนือ อำเภอเมืองภูเก็ต...,"063-082-5566, ...",อุปกรณ์และอะไหล่รถจักรยานยนต์และรถสกูตเตอร์,"https://maps.google.com/maps/place?q=7.879919,..."
6,Naka Loft Cafe,https://www.yellowpages.co.th/profile/Naka-Lof...,155/1 หมู่ที่ 13 ตำบลหนองบัว อำเภอเมืองหนองบัว...,091-405-9888,ร้านของหวาน ไอศกรีม เบเกอรี่,https://maps.google.com/maps/place?q=17.197570...
7,Dog Cafe Bkk - CORGI IN THE GARDEN,https://www.corgiinthegarden.com,,,,
8,Howl 1.1.11 cafe,https://www.yellowpages.co.th/profile/Howl-1-1...,13 ตำบลคอกกระบือ อำเภอเมืองสมุทรสาคร จังหวัดสม...,081-454-5130,โรงแรม บ้านพักและโฮมสเตย์สัตว์เลี้ยง,
9,white house cafe & Studio,https://www.yellowpages.co.th/profile/white-ho...,118 หมู่ที่ 7 ตำบลรอบเมือง อำเภอเมืองร้อยเอ็ด ...,083-288-9939,ร้านของหวาน ไอศกรีม เบเกอรี่,https://maps.google.com/maps/place?q=16.028015...
